# Obtendo informações da rede e dos grupos

In [1]:
from classes import Network, MulticastGroup
from multicastpacking import MulticastPacking, solver
import reader

#instancen of the problem
file = "../../../MPP_instances/n30/b30_1.brite"

links = reader.get_network (file)
net = Network (links, nodes = 30)


mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

problem = MulticastPacking (net, mgroups)

KSIZE = len(problem.groups)+1
NODES = net.nodes+1

# Criando as restrições de Fluxo

## Criando as variáveis do problema: $x_{ij}^{kd}, y_{ij}^{k}$

In [3]:
from gurobipy import *

m = Model ("MPP by Cost")

var_x = {}
var_y = {}

#variable x
for link in net.links:
    for k in xrange (1, KSIZE):
        for d in problem.groups[k-1].members:
            x=link[0],link[1],k,d,
            y=link[1],link[0],k,d,
            var_x[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
            var_x[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            
m.update ()

#variable y
for k in xrange (1, KSIZE):
    for link in net.links:
        x=link[0],link[1],k
        y=link[1],link[0],k
        var_y[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
        var_y[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

m.update ()

## Fluxo 1:  

$\sum_{i \in V} x_{ij}^{kd} - \sum_{i \in V} x_{ji}^{kd} = -1 \quad \forall k \in K,\forall d \in D^k$ 

In [4]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',k,d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,k,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',k,d) )
            == -1,
            name=str(_name)
        )

m.update ()

## Fluxo 2:  

$\sum_{(i,j) \in E} x_{ji}^{kd} - \sum_{(i,j) \in E} x_{ij}^{kd} = 0 \quad \forall k \in K,\forall d \in D^k$ 

In [5]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow2',k,d,j,       
            m.addConstr (
                quicksum(
                   var_x[x] for x in tuplelist (var_x).select ('*',j,k,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',k,d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

m.update ()

## Fluxo 3: 

$\sum_{(i,j) \in E,  j \notin [r_k, d]} x_{ji}^{kd} - \sum_{(i,j) \in E, j \notin [r_k, d]} x_{ji}^{kd} = 0 \quad \forall k \in K,\forall d \in D^k$ 

## Restrição que Força $y_{ij}^{k}$ ser igual 1 se a aresta $(i,j)$ é usada por alguma árvore

In [6]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],k,d,
            y=link[0],link[1],k
            _name='r4',link[0],link[1],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],k,d,
            y=link[1],link[0],k
            _name='r4',link[1],link[0],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )
m.update ()

In [7]:
print (m.update)

<bound method Model.update of <gurobi.Model MIP instance MPP by Cost: 4832 constrs, 4440 vars, Parameter changes: LogFile=gurobi.log>>


# Criando Variável para Capacidade Residual

In [8]:
Z = m.addVar(vtype=GRB.INTEGER, name="Z", obj=1)
m.update ()

## Forçando todos os links terem pelo menos a mesma capacidade Z 

$b_{ij} - \sum_{k \in K} y_{ij}^{k}t^{k} $

In [9]:
for k in xrange(1,KSIZE):
    for link in net.links:
        _name='cap',k,link[0],link[1]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[0],link[1],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
        _name='cap',k,link[1],link[0]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[1],link[0],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
m.update ()        

# Função Objetivo

In [10]:
m.setObjective (Z, GRB.MAXIMIZE)
m.update ()

In [11]:
m.write('Zmodel.lp')

In [12]:
m.optimize ()

Optimize a model with 5432 rows, 4441 columns and 18728 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+01]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+02]
Found heuristic solution: objective 12
Presolve removed 1578 rows and 679 columns
Presolve time: 0.04s
Presolved: 3854 rows, 3762 columns, 12819 nonzeros
Variable types: 0 continuous, 3762 integer (3441 binary)

Root relaxation: objective 3.340000e+01, 928 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.40000    0   18   12.00000   33.40000   178%     -    0s
H    0     0                      25.0000000   33.40000  33.6%     -    0s
H    0     0                      27.0000000   33.40000  23.7%     -    0s
     0     0   33.00000    0   42   27.00000   33.00000  22.2%     -    0s
H    0     0                      28.0000000 

## Imprimindo solução no formato para avaliação

In [13]:
for var in [x for x in tuplelist (var_y).select () if var_y[x].getAttr ('X') == 1.0]:
    #print str(var)+"="+ str(1)
    pass

list =  [x for x in tuplelist (var_y).select () if var_y[x].getAttr ('X') == 1.0]

list = sorted (list, key=lambda tuple: tuple[2] )


with open('out.opt','a') as f:
    for var in list:
        f.write ("%s - %s:%s;\n" % (var[0],var[1],var[2]))
